#Enterprise Fleet Analytics Pipeline: Focuses on the business outcome (analytics) and the domain (fleet/logistics).

![logistics](logistics_project.png)

Download the data from the below gdrive and upload into the catalog
https://drive.google.com/drive/folders/1J3AVJIPLP7CzT15yJIpSiWXshu1iLXKn?usp=drive_link

##**1. Data Munging** -

####1. Visibily/Manually opening the file and capture couple of data patterns (Manual Exploratory Data Analysis)

In [0]:
# Manually when viewed the data in csv file it has Null values in the columns 'hub_location', 'vehicle_type', 'shipment_id', 'role', 'age' ,'first_name', 'last_name' from source2 file
# Manually when viewed the data in csv file it has Null values in the columns 'shipment_id','first_name','last_name','age','role' from source1 file
# Manually when viewed the data in json file it looks clean.
# rawjsondf=spark.read.json("/Volumes/workspace/default/mobile_metrics/logistics_project_data/logistics_shipment_detail_3000.json",multiLine=True)
# rawjsondf.show()

####2. Programatically try to find couple of data patterns applying below EDA (File: logistics_source1)
1. Apply inferSchema and toDF to create a DF and analyse the actual data.
2. Analyse the schema, datatypes, columns etc.,
3. Analyse the duplicate records count and summary of the dataframe.

In [0]:
#Data Exploration programatically
rawdf=spark.read.csv("/Volumes/workspace/default/mobile_metrics/logistics_project_data/logistics_source1",header='true',inferSchema=True).toDF("shipment_id","fname","lname","age","role")
rawdf1=spark.read.csv("/Volumes/workspace/default/mobile_metrics/logistics_project_data/logistics_source2",header='true',inferSchema=True).toDF("shipment_id","fname","lname","age","role","hub_location","vehicle_type")
# display(rawdf.take(10))
# display(rawdf1.take(10))

rawdf.printSchema()
rawdf1.printSchema()

In [0]:
#Source 1
#Important passive Datamunging - EDA data functions we can use
#We identified few patterns on this data
#1. Deduplication of rows and given column(s)
#2. Null values ratio across all columns
#3. Distribution (Dense) of the data across all number columns
#4. Min, Max values
#5. StdDeviation - Standard deviation tells you how much the data varies from the average (mean).
#6. Percentile - Distribution percentage from 0 to 100 in 4 quadrants of 25%
print("actual count of the data",rawdf.count())
print("de-duplicated record (all columns) count",rawdf.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count",rawdf.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given shipmentid column count",rawdf.dropDuplicates(['shipment_id']).count())#de duplicate the entire columns of the given  dataframe
display(rawdf.describe())
display(rawdf.summary())

In [0]:
# Source 2
#Important passive Datamunging - EDA data functions we can use
#We identified few patterns on this data
#1. Deduplication of rows and given column(s)
#2. Null values ratio across all columns
#3. Distribution (Dense) of the data across all number columns
#4. Min, Max values
#5. StdDeviation - Standard deviation tells you how much the data varies from the average (mean).
#6. Percentile - Distribution percentage from 0 to 100 in 4 quadrants of 25%
print("actual count of the data",rawdf1.count())
print("de-duplicated record (all columns) count",rawdf1.distinct().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated record (all columns) count",rawdf1.dropDuplicates().count())#de duplicate the entire columns of the given  dataframe
print("de-duplicated given shipmentid column count",rawdf1.dropDuplicates(['shipment_id']).count())#de duplicate the entire columns of the given  dataframe
display(rawdf1.describe())
display(rawdf1.summary())

###a. Passive Data Munging -  (File: logistics_source1  and logistics_source2)
Without modifying the data, identify:<br>
Shipment IDs that appear in both master_v1 and master_v2<br>
Records where:<br>
1. shipment_id is non-numeric
2. age is not an integer<br>

Count rows having:
3. fewer columns than expected
4. more columns than expected

In [0]:
#Create a Spark Session Object
from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import *
spark=SparkSession.builder.appName("we47_Bread_n_Butter2_Logistics_use_case").getOrCreate()

In [0]:

#Source 1 - Data Cleansing/Munging
print("Source 1")
findshipiddf=rawdf.filter(col("shipment_id").rlike("^[0-9]+$") & (~col("shipment_id").try_cast("int").isNotNull()))
findagedf=rawdf.filter(col("age").isNotNull() & (~col("age").try_cast("int").isNotNull()))
findshipiddf.show()
findagedf.show()
findshipiddf.count()
findagedf.count()
#Source 2 - Data Cleansing/Munging
print("Source 2")
findshipiddf1=rawdf1.filter(col("shipment_id").rlike("^[0-9]+$") & (~col("shipment_id").try_cast("int").isNotNull()))
findagedf1=rawdf1.filter(col("age").isNotNull() & (~col("age").try_cast("int").isNotNull()))
findshipiddf1.show()
findagedf1.show()
findshipiddf1.count()
findagedf1.count()

###**b. Active Data Munging** File: logistics_source1 and logistics_source2

#####1.Combining Data + Schema Merging (Structuring)
1. Read both files without enforcing schema
2. Align them into a single canonical schema: shipment_id,
first_name,
last_name,
age,
role,
hub_location,
vehicle_type,
data_source
3. Add data_source column with values as: system1, system2 in the respective dataframes

In [0]:
from pyspark.sql.functions import lit
strt1="shipment_id int, firstname string, lastname string, age int, role string"
rawdf1=spark.read.schema(strt1).csv(path=["/Volumes/workspace/default/mobile_metrics/logistics_project_data/"],recursiveFileLookup=True,header=True,pathGlobFilter="logistics_source1*")
rawdf1 = rawdf1.withColumn("source", lit("system1"))
strt2="shipment_id int, firstname string, lastname string, age int, role string,hub_loc string, vehicle_type string"
rawdf2=spark.read.schema(strt2).csv(path=["/Volumes/workspace/default/mobile_metrics/logistics_project_data/"],recursiveFileLookup=True,header=True,pathGlobFilter="logistics_source2*")
rawdf2 = rawdf2.withColumn("source", lit("system2"))
#display(rawdf1)
#display(rawdf2)
#Expected right approach to follow
rawdf_merged=rawdf1.unionByName(rawdf2,allowMissingColumns=True)
display(rawdf_merged)
rawdf_merged.write \
    .mode("overwrite") \
    .option("header", False) \
    .csv("/Volumes/workspace/default/mobile_metrics/logistics_project_data/Merged_Data/")



Rejection Strategy

In [0]:
#Before actively Cleansing or Scrubbing - We have to create a Rejection Strategy to reduce data challenges in the future
from pyspark.sql.types import *

strt11 = StructType([
    StructField("shipment_id", IntegerType(), True),
    StructField("firstname", StringType(), True),
    StructField("lastname", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("role", StringType(), True),
    StructField("Source", StringType(), True),
    StructField("Hub_Location", StringType(), True),
    StructField("Vechile_Type", StringType(), True),
    StructField("corruptdata", StringType(), True)
])

rejecteddf = spark.read \
    .schema(strt11) \
    .option("mode", "PERMISSIVE") \
    .option("columnNameOfCorruptRecord", "corruptdata") \
    .csv("/Volumes/workspace/default/mobile_metrics/logistics_project_data/Merged_Data/")
display(rejecteddf)
#Rejected Data
df_reject=rejecteddf .where("corruptdata is not null")
df_reject.drop("corruptdata").write.mode("overwrite").csv("/Volumes/workspace/default/mobile_metrics/logistics_project_data/rejects/",mode="overwrite")
print("Data to reject or update the source",len(rejecteddf.where("corruptdata is not null").collect()))
display(df_reject)

#####2. Cleansing, Scrubbing: 
Cleansing (removal of unwanted datasets)<br>
1. Mandatory Column Check - Drop any record where any of the following columns is NULL:shipment_id, role<br>
2. Name Completeness Rule - Drop records where both of the following columns are NULL: first_name, last_name<br>
3. Join Readiness Rule - Drop records where the join key is null: shipment_id<br>

Scrubbing (convert raw to tidy)<br>
4. Age Defaulting Rule - Fill NULL values in the age column with: -1<br>
5. Vehicle Type Default Rule - Fill NULL values in the vehicle_type column with: UNKNOWN<br>
6. Invalid Age Replacement - Replace the following values in age:
"ten" to -1
"" to -1<br>
7. Vehicle Type Normalization - Replace inconsistent vehicle types: 
truck to LMV
bike to TwoWheeler

In [0]:
print("count of total merged data ",len(rawdf_merged.collect()))
print("count of total merged data after deduplication",rawdf_merged.distinct().count())
cleansed_df=rawdf_merged.na.drop(subset=["shipment_id"]).na.drop(how="all")
cleansed_df=cleansed_df.na.drop(subset=["firstname","lastname"],how='all')
print("Final cleansed DF",len(cleansed_df.collect()))
display(cleansed_df)
print("count of cleansed data after removing null values",cleansed_df.count())

In [0]:
cleansed_df.printSchema()
#scrubbing the data
#scrubbed_df = cleansed_df.na.fill(-1, subset=["age"])
scrubbed_df1 = (
    cleansed_df
        .na.fill(-1, subset=["age"])                # age is numeric → use -1 (int)
        .na.fill("UNKNOWN", subset=["vehicle_type"])  
)

#display(scrubbed_df1)
scrubbed_df2=scrubbed_df1.na.fill(-1, subset=["age"])
#display(scrubbed_df2)
dict1 = {
    "Truck": "LMV",
    "Bike": "Two Wheeler"
}
scrubbed_df = scrubbed_df2.na.replace(
    dict1,
    subset=["vehicle_type"]
)

print("scrubbed DF count:", scrubbed_df.count())
display(scrubbed_df)
scrubbed_df.count()


####3. Standardization, De-Duplication and Replacement / Deletion of Data to make it in a usable format

Creating shipments Details data Dataframe creation <br>
1. Create a DF by Reading Data from logistics_shipment_detail.json
2. As this data is a clean json data, it doesn't require any cleansing or scrubbing.

In [0]:
shipmentdf = spark.read.json("/Volumes/workspace/default/mobile_metrics/logistics_project_data/logistics_shipment_detail_3000.json",multiLine=True)
display(shipmentdf)

Standardizations:<br>

1. Add a column<br> 
Source File: DF of logistics_shipment_detail_3000.json<br>: domain as 'Logistics',  current timestamp 'ingestion_timestamp' and 'False' as 'is_expedited'
2. Column Uniformity: 
role - Convert to lowercase<br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
vehicle_type - Convert values to UPPERCASE<br>
Source Files: DF of logistics_shipment_detail_3000.json
hub_location - Convert values to initcap case<br>
Source Files: DF of merged(logistics_source1 & logistics_source2)<br>
3. Format Standardization:<br>
Source Files: DF of logistics_shipment_detail_3000.json<br>
Convert shipment_date to yyyy-MM-dd<br>
Ensure shipment_cost has 2 decimal precision<br>
4. Data Type Standardization<br>
Standardizing column data types to fix schema drift and enable mathematical operations.<br>
Source File: DF of merged(logistics_source1 & logistics_source2) <br>
age: Cast String to Integer<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
shipment_weight_kg: Cast to Double<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
is_expedited: Cast to Boolean<br>
5. Naming Standardization <br>
Source File: DF of merged(logistics_source1 & logistics_source2)<br>
Rename: first_name to staff_first_name<br>
Rename: last_name to staff_last_name<br>
Rename: hub_location to origin_hub_city<br>
6. Reordering columns logically in a better standard format:<br>
Source File: DF of Data from all 3 files<br>
shipment_id (Identifier), staff_first_name (Dimension)staff_last_name (Dimension), role (Dimension), origin_hub_city (Location), shipment_cost (Metric), ingestion_timestamp (Audit)

In [0]:
from pyspark.sql.functions import lit, current_timestamp, initcap, upper, col, to_date, expr
from pyspark.sql.types import DecimalType

df_enriched = shipmentdf.withColumn("domain", lit("Logistics")) \
    .withColumn("ingestion_timestamp", current_timestamp()) \
    .withColumn("is_expedited", lit(False))

df_enriched = df_enriched.withColumn("vehicle_type",upper(col("vehicle_type")))
scrubbed_df = scrubbed_df.withColumn("vehicle_type",upper(col("vehicle_type"))).withColumn("hub_loc",initcap(col("hub_loc")))

df_enriched = df_enriched.withColumn(
    "shipment_date",
    expr("try_to_date(shipment_date, 'dd-MM-yy')")#when used as  to_date(col("shipment_date"), "yyyy-MM-dd")) Error CANNOT_PARSE_TIMESTAMP] Text '24-04-23' could not be parsed at index 0. Use try_to_date to tolerate invalid input string and return NULL instead. SQLSTATE: 22007. Hence to avoid pipeline failure due to bad dates, use try_to_date
).withColumn(
    "shipment_cost",
    col("shipment_cost").cast(DecimalType(10, 2))
)
df_enriched = df_enriched.withColumn(
    "shipment_weight_kg",
     col("shipment_weight_kg").cast(DoubleType())
).withColumn(
    "is_expedited",
    col("is_expedited").cast("boolean")
)
#Naming Standardization
scrubbed_df = (
    scrubbed_df
    .withColumnRenamed("firstname", "staff_first_name")
    .withColumnRenamed("lastname", "staff_last_name")
    .withColumnRenamed("hub_loc","origin_hub_city")
)
#df_enriched.printSchema()

scrubbed_df = scrubbed_df.select("shipment_id","staff_first_name","staff_last_name","age","role","origin_hub_city","vehicle_type")
display(scrubbed_df)
df_enriched = df_enriched.select("shipment_id","shipment_date","source_city","destination_city","vehicle_type","shipment_cost","shipment_weight_kg","domain","ingestion_timestamp","is_expedited")
display(df_enriched)

Deduplication:
1. Apply Record Level De-Duplication
2. Apply Column Level De-Duplication (Primary Key Enforcement)

In [0]:
dedup_df=scrubbed_df.dropDuplicates(subset=["shipment_id"])
display(dedup_df)
dedup_df.count()
dedup_df=dedup_df.dropDuplicates(["shipment_id","staff_first_name","staff_last_name","age"])
dedup_df.count()

##2. Data Enrichment - Detailing of data
Makes your data rich and detailed <br>

###### Adding of Columns (Data Enrichment)
*Creating new derived attributes to enhance traceability and analytical capability.*

**1. Add Audit Timestamp (`load_dt`)**
Source File: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** We need to track exactly when this record was ingested into our Data Lakehouse for auditing purposes.
* **Action:** Add a column `load_dt` using the function `current_timestamp()`.

**2. Create Full Name (`full_name`)**
Source File: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** The reporting dashboard requires a single field for the driver's name instead of separate columns.
* **Action:** Create `full_name` by concatenating `first_name` and `last_name` with a space separator.
* **Result:** "Rajesh" + " " + "Kumar" -> **"Rajesh Kumar"**

**3. Define Route Segment (`route_segment`)**
Source File: DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** The logistics team wants to analyze performance based on specific transport lanes (Source to Destination).
* **Action:** Combine `source_city` and `destination_city` with a hyphen.
* **Result:** "Chennai" + "-" + "Pune" -> **"Chennai-Pune"**

**4. Generate Vehicle Identifier (`vehicle_identifier`)**
Source File: DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** We need a unique tracking code that immediately tells us the vehicle type and the shipment ID.
* **Action:** Combine `vehicle_type` and `shipment_id` to create a composite key.
* **Result:** "Truck" + "_" + "500001" -> **"Truck_500001"**

In [0]:
from pyspark.sql.functions import concat, col, lit
dedup_df=dedup_df.withColumn("load_dt", current_timestamp())
enrichsrc1_df=dedup_df.withColumn(
    "staff_full_name",
    concat(col("staff_first_name"), lit(" "), col("staff_last_name"))
)
enrichsrc1_df=enrichsrc1_df.drop("staff_first_name","staff_last_name").select("shipment_id","staff_full_name","age","role","origin_hub_city","vehicle_type","load_dt")
df_enriched = df_enriched.withColumn(
    "Source to Destination",
    concat(col("source_city"), lit(" - "), col("destination_city"))
)
df_enriched=df_enriched.drop("source_city","destination_city").select("shipment_id","shipment_date","vehicle_type","shipment_cost","shipment_weight_kg","domain","ingestion_timestamp","is_expedited","Source to Destination")
df_enriched= df_enriched.withColumn(
    "tracking_code",
    concat_ws("_", col("vehicle_type"), col("shipment_id").cast("string"))
)

display(df_enriched.select("vehicle_type", "shipment_id", "tracking_code").limit(10))
display(enrichsrc1_df)

###### Deriving of Columns (Time Intelligence)
*Extracting temporal features from dates to enable period-based analysis and reporting.*<br>
Source File: logistics_shipment_detail_3000.json<br>
**1. Derive Shipment Year (`shipment_year`)**
* **Scenario:** Management needs an annual performance report to compare growth year-over-year.
* **Action:** Extract the year component from `shipment_date`.
* **Result:** "2024-04-23" -> **2024**

**2. Derive Shipment Month (`shipment_month`)**
* **Scenario:** Analysts want to identify seasonal peaks (e.g., increased volume in December).
* **Action:** Extract the month component from `shipment_date`.
* **Result:** "2024-04-23" -> **4** (April)

**3. Flag Weekend Operations (`is_weekend`)**
* **Scenario:** The Operations team needs to track shipments handled during weekends to calculate overtime pay or analyze non-business day capacity.
* **Action:** Flag as **'True'** if the `shipment_date` falls on a Saturday or Sunday.

**4. Flag shipment status (`is_expedited`)**
* **Scenario:** The Operations team needs to track shipments is IN_TRANSIT or DELIVERED.
* **Action:** Flag as **'True'** if the `shipment_status` IN_TRANSIT or DELIVERED.

In [0]:
from pyspark.sql.functions import year, col, when, lit
df_enriched = df_enriched.withColumn("shipment_year", year(col("shipment_date")))
df_enriched = df_enriched.withColumn("shipment_month", month(col("shipment_date")))
df_shipment = df_enriched.withColumn("is_weekend",when((dayofweek(col("shipment_date"))==1) | (dayofweek(col("shipment_date"))==7),"True").otherwise("False"))
df_shipment=df_shipment.withColumn(
    "is_expedited",
    when(col("shipment_status").isin("IN_TRANSIT", "DELIVERED"), lit(True))
    .otherwise(lit(False))
)
display(df_shipment)

###### Enrichment/Business Logics (Calculated Fields)
*Deriving new metrics and financial indicators using mathematical and date-based operations.*<br>
Source File: logistics_shipment_detail_3000.json<br>

**1. Calculate Unit Cost (`cost_per_kg`)**
* **Scenario:** The Finance team wants to analyze the efficiency of shipments by determining the cost incurred per unit of weight.
* **Action:** Divide `shipment_cost` by `shipment_weight_kg`.
* **Logic:** `shipment_cost / shipment_weight_kg`

**2. Track Shipment Age (`days_since_shipment`)**
* **Scenario:** The Operations team needs to monitor how long it has been since a shipment was dispatched to identify potential delays.
* **Action:** Calculate the difference in days between the `current_date` and the `shipment_date`.
* **Logic:** `datediff(current_date(), shipment_date)`

**3. Compute Tax Liability (`tax_amount`)**
* **Scenario:** For invoicing and compliance, we must calculate the Goods and Services Tax (GST) applicable to each shipment.
* **Action:** Calculate 18% GST on the total `shipment_cost`.
* **Logic:** `shipment_cost * 0.18`

In [0]:
from pyspark.sql.functions import col, when, current_date, datediff
from pyspark.sql.types import DecimalType

enrichship_df = (
    shipmentdf
    # Unit cost
    .withColumn(
        "cost_per_kg",
        when(
            col("shipment_weight_kg") > 0,
            col("shipment_cost") / col("shipment_weight_kg")
        )
    )
    # Shipment age
    .withColumn(
        "days_since_shipment",
        datediff(current_date(), col("shipment_date"))
    )
    # Tax amount (18%)
    .withColumn(
        "tax_amount",
        (col("shipment_cost") * 0.18).cast(DecimalType(10, 2))
    )
)


###### Remove/Eliminate (drop, select, selectExpr)
*Excluding unnecessary or redundant columns to optimize storage and privacy.*<br>
Source File: DF of logistics_source1 and logistics_source2<br>

**1. Remove Redundant Name Columns**
* **Scenario:** Since we have already created the `full_name` column in the Enrichment step, the individual name columns are now redundant and clutter the dataset.
* **Action:** Drop the `first_name` and `last_name` columns.
* **Logic:** `df.drop("first_name", "last_name")`

##### Splitting & Merging/Melting of Columns
*Reshaping columns to extract hidden values or combine fields for better analysis.*<br>
Source File: DF of logistics_shipment_detail_3000.json<br>
**1. Splitting (Extraction)**
*Breaking one column into multiple to isolate key information.*
* **Split Order Code:**
  * **Action:** Split `order_id` ("ORD100000") into two new columns:
    * `order_prefix` ("ORD")
    * `order_sequence` ("100000")
* **Split Date:**
  * **Action:** Split `shipment_date` into three separate columns for partitioning:
    * `ship_year` (2024)
    * `ship_month` (4)
    * `ship_day` (23)

**2. Merging (Concatenation)**
*Combining multiple columns into a single unique identifier or description.*
* **Create Route ID:**
  * **Action:** Merge `source_city` ("Chennai") and `destination_city` ("Pune") to create a descriptive route key:
    * `route_lane` ("Chennai->Pune")

## 3. Data Customization & Processing - Application of Tailored Business Specific Rules

### **UDF1: Complex Incentive Calculation**
**Scenario:** The Logistics Head wants to calculate a "Performance Bonus" for drivers based on tenure and role complexity.

**Action:** Create a Python function `calculate_bonus(role, age)` and register it as a Spark UDF.

**Logic:**
* **IF** `Role` == 'Driver' **AND** `Age` > 50:
  * `Bonus` = 15% of Salary (Reward for Seniority)
* **IF** `Role` == 'Driver' **AND** `Age` < 30:
  * `Bonus` = 5% of Salary (Encouragement for Juniors)
* **ELSE**:
  * `Bonus` = 0

**Result:** A new derived column `projected_bonus` is generated for every row in the dataset.

---

### **UDF2: PII Masking (Privacy Compliance)**
**Scenario:** For the analytics dashboard, we must hide the full identity of the staff to comply with privacy laws (GDPR/DPDP), while keeping names recognizable for internal managers.

**Business Rule:** Show the first 2 letters, mask the middle characters with `****`, and show the last letter.

**Action:** Create a UDF `mask_identity(name)`.

**Example:**
* **Input:** `"Rajesh"`
* **Output:** `"Ra****h"`
<br>
**Note: Convert the above udf logic to inbult function based transformation to ensure the performance is improved.**

## 4. Data Core Curation & Processing (Pre-Wrangling)
*Applying business logic to focus, filter, and summarize data before final analysis.*

**1. Select (Projection)**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** The Driver App team only needs location data, not sensitive HR info.
* **Action:** Select only `first_name`, `role`, and `hub_location`.

**2. Filter (Selection)**<br>
Source File: DF of json<br>
* **Scenario:** We need a report on active operational problems.
* **Action:** Filter rows where `shipment_status` is **'DELAYED'** or **'RETURNED'**.
* **Scenario:** Insurance audit for senior staff.
* **Action:** Filter rows where `age > 50`.

**3. Derive Flags & Columns (Business Logic)**<br>
Source File: DF of json<br>
* **Scenario:** Identify high-value shipments for security tracking.
* **Action:** Create flag `is_high_value` = **True** if `shipment_cost > 50,000`.
* **Scenario:** Flag weekend operations for overtime calculation.
* **Action:** Create flag `is_weekend` = **True** if day is Saturday or Sunday.

**4. Format (Standardization)**<br>
Source File: DF of json<br>
* **Scenario:** Finance requires readable currency formats.
* **Action:** Format `shipment_cost` to string like **"₹30,695.80"**.
* **Scenario:** Standardize city names for reporting.
* **Action:** Format `source_city` to Uppercase (e.g., "chennai" → **"CHENNAI"**).

**5. Group & Aggregate (Summarization)**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Scenario:** Regional staffing analysis.
* **Action:** Group by `hub_location` and **Count** the number of staff.
* **Scenario:** Fleet capacity analysis.
* **Action:** Group by `vehicle_type` and **Sum** the `shipment_weight_kg`.

**6. Sorting (Ordering)**<br>
Source File: DF of json<br>
* **Scenario:** Prioritize the most expensive shipments.
* **Action:** Sort by `shipment_cost` in **Descending** order.
* **Scenario:** Organize daily dispatch schedule.
* **Action:** Sort by `shipment_date` (Ascending) then `priority_flag` (Descending).

**7. Limit (Top-N Analysis)**<br>
Source File: DF of json<br>
* **Scenario:** Dashboard snapshot of critical delays.
* **Action:** Filter for 'DELAYED', Sort by Cost, and **Limit to top 10** rows.

## 5. Data Wrangling - Transformation & Analytics
*Combining, modeling, and analyzing data to answer complex business questions.*

### **1. Joins**
Source Files:<br>
Left Side (staff_df):<br> DF of logistics_source1 & logistics_source2<br>
Right Side (shipments_df):<br> DF of logistics_shipment_detail_3000.json<br>
#### **1.1 Frequently Used Simple Joins (Inner, Left)**
* **Inner Join (Performance Analysis):**
  * **Scenario:** We only want to analyze *completed work*. Connect Staff to the Shipments they handled.
  * **Action:** Join `staff_df` and `shipments_df` on `shipment_id`.
  * **Result:** Returns only rows where a staff member is assigned to a valid shipment.
* **Left Join (Idle Resource check):**
  * **Scenario:** Find out which staff members are currently *idle* (not assigned to any shipment).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right) on `shipment_id`. Filter where `shipments_df.shipment_id` is NULL.

#### **1.2 Infrequent Simple Joins (Self, Right, Full, Cartesian)**
* **Self Join (Peer Finding):**
  * **Scenario:** Find all pairs of employees working in the same `hub_location`.
  * **Action:** Join `staff_df` to itself on `hub_location`, filtering where `staff_id_A != staff_id_B`.
* **Right Join (Orphan Data Check):**
  * **Scenario:** Identify shipments in the system that have *no valid driver* assigned (Data Integrity Issue).
  * **Action:** Join `staff_df` (Left) with `shipments_df` (Right). Focus on NULLs on the left side.
* **Full Outer Join (Reconciliation):**
  * **Scenario:** A complete audit to find *both* idle drivers AND unassigned shipments in one view.
  * **Action:** Perform a Full Outer Join on `shipment_id`.
* **Cartesian/Cross Join (Capacity Planning):**
  * **Scenario:** Generate a schedule of *every possible* driver assignment to *every* pending shipment to run an optimization algorithm.
  * **Action:** Cross Join `drivers_df` and `pending_shipments_df`.

#### **1.3 Advanced Joins (Semi and Anti)**
* **Left Semi Join (Existence Check):**
  * **Scenario:** "Show me the details of Drivers who have *at least one* shipment." (Standard filtering).
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_semi")`.
  * **Benefit:** Performance optimization; it stops scanning the right table once a match is found.
* **Left Anti Join (Negation Check):**
  * **Scenario:** "Show me the details of Drivers who have *never* touched a shipment."
  * **Action:** `staff_df.join(shipments_df, "shipment_id", "left_anti")`.

### **2. Lookup**<br>
Source File: DF of logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Validation. Check if the `hub_location` in the staff file exists in the corporate `Master_City_List`.
* **Action:** Compare values against a reference list.

### **3. Lookup & Enrichment**<br>
Source File: DF of logistics_source1 and logistics_source2 (merged into Staff DF)<br>
* **Scenario:** Geo-Tagging.
* **Action:** Lookup `hub_location` ("Pune") in a Master Latitude/Longitude table and enrich the dataset by adding `lat` and `long` columns for map plotting.

### **4. Schema Modeling (Denormalization)**<br>
Source Files: DF of All 3 Files (logistics_source1, logistics_source2, logistics_shipment_detail_3000.json)<br>
* **Scenario:** Creating a "Gold Layer" Table for PowerBI/Tableau.
* **Action:** Flatten the Star Schema. Join `Staff`, `Shipments`, and `Vehicle_Master` into one wide table (`wide_shipment_history`) so analysts don't have to perform joins during reporting.

### **5. Windowing (Ranking & Trends)**<br>
Source Files:<br>
DF of logistics_source2: Provides hub_location (Partition Key).<br>
logistics_shipment_detail_3000.json: Provides shipment_cost (Ordering Key)<br>
* **Scenario:** "Who are the Top 3 Drivers by Cost in *each* Hub?"
* **Action:**
  1. Partition by `hub_location`.
  2. Order by `total_shipment_cost` Descending.
  3. Apply `dense_rank()` and `row_number()
  4. Filter where `rank or row_number <= 3`.

### **6. Analytical Functions (Lead/Lag)**<br>
Source File: <br>
DF of logistics_shipment_detail_3000.json<br>
* **Scenario:** Idle Time Analysis.
* **Action:** For each driver, calculate the days elapsed since their *previous* shipment.

### **7. Set Operations**<br>
Source Files: DF of logistics_source1 and logistics_source2<br>
* **Union:** Combining `Source1` (Legacy) and `Source2` (Modern) into one dataset (Already done in Active Munging).
* **Intersect:** Identifying Staff IDs that appear in *both* Source 1 and Source 2 (Duplicate/Migration Check).
* **Except (Difference):** Identifying Staff IDs present in Source 2 but *missing* from Source 1 (New Hires).

### **8. Grouping & Aggregations (Advanced)**<br>
Source Files:<br>
DF of logistics_source2: Provides hub_location and vehicle_type (Grouping Dimensions).<br>
DF of logistics_shipment_detail_3000.json: Provides shipment_cost (Aggregation Metric).<br>
* **Scenario:** The CFO wants a subtotal report at multiple levels:
  1. Total Cost by Hub.
  2. Total Cost by Hub AND Vehicle Type.
  3. Grand Total.
* **Action:** Use `cube("hub_location", "vehicle_type")` or `rollup()` to generate all these subtotals in a single query.

##6. Data Persistance (LOAD)-> Data Publishing & Consumption<br>

Store the inner joined, lookup and enrichment, Schema Modeling, windowing, analytical functions, set operations, grouping and aggregation data into the delta tables.

##7.Take the copy of the above notebook and try to write the equivalent SQL for which ever applicable.